# Fetching Data

This file is pulling all the necessary informations from the github repositories and formatting it into the following JSON format:
- Name of Issue
- ID of the Issue
- Name of the Repo
- ID of the Repo
- Code Changes

In [1]:
# Parameters for the Notebook
# You can set the parameters to the ones you need before running the entire notebook

github_url: str = ""                # url of the repo, set only if you want a manual issue
manual_issue: bool = False          # set to True if you have a specific url to run
first_run: bool = False             # set to False once you ran the entire notebook once already
path_for_course: str = ""           # String with the full path where the course folder should be created

# Set-Up of the Notebook

import pandas as pd
import numpy as np
import os
from github import Github
from dotenv import load_dotenv

load_dotenv()
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_OBJECT = Github(GITHUB_TOKEN)




### Creating the Database

We want to select the proper issues from all the top repositories to create the courses from. Thus we are looking for issues that match all the following criteria:

- Is from a JS/TS repository
- Is a closed issue
- Is a good first issue
- Only has one pull request
- Is an issue that only makes code changes

The reasoning behind those criteria is on one hand, to have a relevant coding issue to give to students who work with open source software for the first time, and on the other hand to make it easier for us to compute all the exercises.

In [2]:
reactRepos = GITHUB_OBJECT.search_repositories(query=f'topic:react language:JavaScript language:TypeScript', sort="stars", order="desc")

In [10]:
def select_issues(manual_issue: bool, repos, issue_url: str):
    """
    This function outputs a DataFrame consisting of all issues that follow the following criteria
    - Only good first issues
    - Only issues that don't have something to do with the documentation
    """

    filtered_words = ["doc", "document", "DOC", "Doc", "Document", "docs", "Docs", "DOCS", "Docs", "style", "readme", "README"]
    issue_list = []

    if not manual_issue:
        for inx, repo in enumerate(repos):

            # if repo.size > 100000 or repo.size <= 50000:
            #     continue

            issues = repo.get_issues(state="closed", labels=["good first issue"])
            print(issues.size)
            for indx, issue in enumerate(issues):
                flag: bool = False
                if issue.pull_request != None:
                    continue
                for word in filtered_words:
                    if word in issue.title:
                        flag = True
                        break 
                if flag:
                    break

                issue_list.append({"name": issue.title, "repo": repo, "issue": [issue]})

            if inx == 99:
                break

        issue_dataframe = pd.DataFrame(issue_list)
        issue_dataframe.to_csv(path_or_buf="data/selected_issues.csv")
        
        return issue_dataframe

selected_issues = select_issues(manual_issue, reactRepos, github_url)

0
Issue(title="Issue on Timestamp Microservice @ Back End Development and APIs in Test Case 7 and 8", number=60459)
Issue(title="House Painting - odd requirement for the window placement test", number=60458)
Issue(title="Movie Review Page - imprecise requirements", number=60457)
Issue(title="fix: update Task 71 expected answer from 'speeding' to 'speeding up'", number=60456)
Issue(title="Fix(curriculum):Fix background gradient test regex for accurate valid…", number=60455)
Issue(title="fix(curriculum): correct inaccurate answer in express agreement challenge ", number=60454)
Issue(title="chore(client): update curriculum-helpers to 4.1.0", number=60453)
Issue(title="Feature Request: GitHub Integration for Auto-Saving Exercise Progress", number=60452)
Issue(title="feat(curriculum): Add A2 Chinese Outline", number=60451)
Issue(title="chore(curriculum): remove concatenation asserts in Trivia Bot lab", number=60450)
Issue(title="Python Terminal not Accessible via Screen Reader", number=6044

KeyboardInterrupt: 

In [36]:
def process_issues(first_run: bool, manual_issue: bool, issues):
    """
    This function outputs a dataframe with a list of some the selected issues, all filtered 
    and formatted properly according to our schema:
    {
        repo:   # Repository the issue belongs to
        title:  # The title of the issue
        issue:  # The issue itself
        pr:     # The list of events in the pull request
        a:      # Title of the issue the events belong to
    }
    """
    
    filtered_issues = []
    counter = 0
    debug_counter = 0

    if not manual_issue:
        for indx, row in issues.iterrows():
            timeline = row["issue"][0].get_timeline()

            num_pr = 0
            list_pr = []
            list_a = []

            # Look for the cross-referenced pull request of the issue
            for event in timeline:
                if event.event == "cross-referenced" and event.source.issue.pull_request and event.source.issue.pull_request != None:
                    print("Got in!")
                    list_pr.append({"pr": event.source.issue, "event": event})
                    list_a.append({"a": repo["name"]})
                    num_pr += 1
                if num_pr > 1:
                    debug_counter -= num_pr
                    break

            if num_pr == 1:
                filtered_issues.append({
                    "repo": row["repo"],
                    "title": row["name"],
                    "issue": row["issue"],
                    "pr": list_pr,
                    "a": list_a
                })
                counter += 1
                print(counter)

            if counter > 99:
                break
        
        filtered_issues_dataframe = pd.DataFrame(filtered_issues)
        filtered_issues_dataframe.to_csv(path_or_buf="data/filtered_issues.csv")
    
        return filtered_issues_dataframe

filtered_issues = process_issues(True, manual_issue, selected_issues)

In [ ]:
def get_coding_changes(first_run: bool, manual_issue: bool, data):
    if not manual_issue:
        updated_data = []

        return updated_data_dataframe

updated_data_dataframe = get_coding_changes(first_run, manual_issue, filtered_issues)

### Creating the Courses' File Structure

JetBrains have a specific file structure which allows us to generate courses just by following it. The key part of the JetBrains Academy course structure are the yaml files that define what this specific folder should do. There are 3 types a folder can be, with some types having subtypes:

- **Course:** The folder is encompassing the entire course. In there we define what the content of the course is and what the additional files are.
- **Lessons:** 
    - *Lesson*: Normal lessons where each task is independent from each other
    - *Guided Project*: Lesson with the `type: framework` tag, propagating the file changes made by the user from one task to the other
- **Tasks**:
    -**

### Create Course Input Format

The format of the create_course function is a json file with the following structure.

```text
{
    "title": "Course Title",
    "lessons": [
        {
            "title": "Lesson Title",
            "type": "edu or nothing",
            "tasks": [
                {
                    "title": "Task Title",
                    "type": "type of the task",
                    "description": "Generated Description of the Task"
                }
            ]
        }
    ],
    "additional content"
}
```

In [ ]:
def create_course(course_data):
    """
    This function takes a JSON file as an input, and uses this input to create the entire folder structure
    for the JetBrains Academy course
    """

    